In [1]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc
import pandas as pd
import xgboost as xgb
from sklearn import metrics
import numpy as np
from sklearn.grid_search import GridSearchCV

/Users/macbookproi7/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/macbookproi7/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train_df = pd.read_csv('/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/train_data.csv')
test_df = pd.read_csv('/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/test_data.csv')
train=train_df.copy()
train.set_index("id", inplace=True)
test=test_df.copy()
test.set_index("id",inplace=True)
train.fillna(-1, inplace=True) ##### how should you deal with Na's
test.fillna(-1,inplace=True)

train_float_int = train.select_dtypes(include=['float64']) ##To help create dummy variables
train_obj= train.select_dtypes(include=['O'])
test_float_int=test.select_dtypes(include=['float64'])
test_obj=test.select_dtypes(include=['O'])
train_dummies= pd.get_dummies(train_obj) ###Creates dummies (Not supposed to do! figure out)
test_dummies = pd.get_dummies(test_obj)
train = pd.concat([train_float_int,train_dummies],axis=1)
test=pd.concat([test_float_int, test_dummies],axis=1)
train.fillna(train.mean(), inplace=True)
test.fillna(train.mean(),inplace=True)
print("nothing")

nothing


In [3]:
from sklearn.model_selection import train_test_split
y = train["target"]
X= train.drop("target",1)

#X =  train.drop("target", 1) 
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=61,
                                                    stratify = y)

In [4]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.013661359688577634, 'n_estimators': 305, 'seed':24, 'subsample': 0.63399548138500328, 'colsample_bytree': 0.80741756013161081, 
             'objective': 'binary:logistic'}
optimized_XGB = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'roc_auc', cv = 5, n_jobs = -1) 

In [5]:
optimized_XGB.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1,
       colsample_bytree=0.8074175601316108, gamma=0,
       learning_rate=0.013661359688577634, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=305, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=24, silent=True,
       subsample=0.6339954813850033),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [6]:
optimized_XGB.grid_scores_

[mean: 0.62755, std: 0.00252, params: {'max_depth': 3, 'min_child_weight': 1},
 mean: 0.62755, std: 0.00252, params: {'max_depth': 3, 'min_child_weight': 3},
 mean: 0.62743, std: 0.00242, params: {'max_depth': 3, 'min_child_weight': 5},
 mean: 0.63222, std: 0.00155, params: {'max_depth': 5, 'min_child_weight': 1},
 mean: 0.63205, std: 0.00147, params: {'max_depth': 5, 'min_child_weight': 3},
 mean: 0.63211, std: 0.00138, params: {'max_depth': 5, 'min_child_weight': 5},
 mean: 0.63235, std: 0.00117, params: {'max_depth': 7, 'min_child_weight': 1},
 mean: 0.63209, std: 0.00144, params: {'max_depth': 7, 'min_child_weight': 3},
 mean: 0.63180, std: 0.00111, params: {'max_depth': 7, 'min_child_weight': 5}]

In [7]:
y_estimated_test = optimized_XGB.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test, y_estimated_test)

0.63357479969267916

In [8]:
# submission_example = pd.DataFrame({"prediction": optimized_XGB.predict_proba(test)[:, 1]}, index=test.index) 
# submission_example.to_csv("./submission_example3.csv")


submission_example = pd.DataFrame({"prediction": optimized_XGB.predict_proba(test)[:, 1]}, index=X_test.index)
submission_example.to_csv("./submission_example.csv")

ValueError: Shape of passed values is (1, 19525), indices imply (1, 9112)

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics

YOUR_FILE = "/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/submission_example.csv"
TEST_FILE = "/Users/macbookproi7/Desktop/PHYS 590/590/homework/homework5/test_data.csv"

submission_df = pd.read_csv(YOUR_FILE, index_col=["id"])
test_df = pd.read_csv(TEST_FILE, index_col=["id"])

fake_test_target = pd.DataFrame({"target": np.random.choice([0,1], size=test_df.shape[0])}, index=test_df.index)
submission_df["Truth"] = fake_test_target["target"]

metrics.roc_auc_score(submission_df["Truth"], submission_df["prediction"])